# Официальный производственный календарь РФ с 2013 года

## Таблица дат

Источник - http://xmlcalendar.ru/

тип дня (type): 1 - выходной день, 
                2 - рабочий и сокращенный (может быть использован для любого дня недели), 
                3 - рабочий день (если суббота/воскресенье)

In [33]:
import xml.etree.ElementTree as ET
import pandas as pd
import requests
from datetime import date

# URL шаблон
url_template = "http://xmlcalendar.ru/data/ru/{}/calendar.xml"

# Годы для загрузки
start_year = 2013
current_year = date.today().year
years = range(start_year, current_year + 2)  # Включаем текущий и следующий год
# Как правило, данные по следующему году подгружаются осенью текущего

# Функция для проверки существования файла
def file_exists(url):
    response = requests.head(url)
    return response.status_code == 200

# Названия праздников из текущего года
response = requests.get(url_template.format(current_year))
xml_data = response.content
root = ET.fromstring(xml_data)

holidays = []
for holiday in root.find('holidays'):
    holidays.append({
        'id': holiday.get('id'),
        'title': holiday.get('title')
    })

holidays_df = pd.DataFrame(holidays)
holidays_df.id = holidays_df.id.astype(str).astype(int)

# Данные по всем праздничным дням и переносам рабочих дней с 2013 года
all_days = []

for year in years:
    # Загрузка содержимого XML по URL
    url = url_template.format(year)
    if file_exists(url):
        print('Загрузка данных:', year, end='\r')
        response = requests.get(url)
        xml_data = response.content

        # Парсим XML-документ
        root = ET.fromstring(xml_data)

        # Извлекаем данные по каждому дню в all_days
        for day in root.find('days'):
            day_str = day.get('d')
            day_parts = day_str.split('.')
            day_date = date(year, int(day_parts[0]), int(day_parts[1]))
            all_days.append({
                'Date': day_date,
                'type': day.get('t'),
                'title_id': day.get('h', ''),  # Заполняем пустыми строками, если атрибут отсутствует
                'from_day': day.get('f', '')   # Заполняем пустыми строками, если атрибут отсутствует
            })
                        
# Создаем DataFrame days
days_df = pd.DataFrame(all_days)
print('Загружено дней:', len(days_df.type), '    ')

#Приводим в соответствие типы данных
days_df.Date = pd.to_datetime(days_df.Date, yearfirst=True)
days_df.type = pd.to_numeric(days_df.type, errors='raise').convert_dtypes()
days_df.title_id = pd.to_numeric(days_df.title_id, errors='raise').convert_dtypes()

#Сохраняем названия праздников и список дней в файлы csv
holidays_df.to_csv('holidays_titles.csv', index=False)
days_df.to_csv('holidays_list.csv', index=False)
print('Данные по праздничным дням сохранены в файлы')


#Создание таблицы дат
date_list = [date(start_year, 1, 1) + timedelta(days=x) for x in 
             range((date(days_df.Date[-1:].dt.year.item(), 12, 31) - date(start_year, 1, 1)).days + 1)]
dates_df = pd.DataFrame(date_list, columns=['Date'])
dates_df.Date = pd.to_datetime(dates_df.Date, yearfirst=True)

#создаём DataFrame dates_df
dates_df = dates_df.merge(days_df, on='Date', how='left')
status_list = []
for day, t in zip(dates_df.Date.fillna(0).to_list(), dates_df.type.fillna(0).to_list()):
    if t == 1:
        status_list.append('Праздничный')
        continue
    elif t == 2:
        status_list.append('Рабочий')
        continue     
    elif t == 3:
        status_list.append('Рабочий')
        continue
    elif day.weekday() == 5 or day.weekday() == 6:
        status_list.append('Выходной')
        continue
    else:
        status_list.append('Рабочий')
dates_df['status'] = status_list
dates_df['year'] = pd.DatetimeIndex(dates_df.Date).year
dates_df['year_iso'] = pd.DatetimeIndex(dates_df.Date).isocalendar().year.to_list()
dates_df['quarter'] = pd.DatetimeIndex(dates_df.Date).quarter
dates_df['month'] = pd.DatetimeIndex(dates_df.Date).month
dates_df['week_iso'] = pd.DatetimeIndex(dates_df.Date).isocalendar().week.to_list()
dates_df['day_of_month'] = pd.DatetimeIndex(dates_df.Date).day
dates_df['day_of_week'] = pd.DatetimeIndex(dates_df.Date).isocalendar().day.to_list()

#Сохраняем таблицу дат в файл csv
dates_df.to_csv('dates_table.csv', index=False)
print('Таблица дат сохранена в файл')

Загружено дней: 323     
Данные по праздничным дням сохранены в файлы
Таблица дат сохранена в файл


In [35]:
holidays_df

,id,title
0,1,Новогодние каникулы
1,2,Рождество Христово
2,3,День защитника Отечества
3,4,Международный женский день
4,5,Праздник Весны и Труда
5,6,День Победы
6,7,День России
7,8,День народного единства


In [36]:
days_df

,Date,type,title_id,from_day
0,2013-01-01,1,1,
1,2013-01-02,1,1,
2,2013-01-03,1,1,
3,2013-01-04,1,1,
4,2013-01-05,1,1,
...,...,...,...,...
318,2024-11-02,2,<NA>,
319,2024-11-04,1,8,
320,2024-12-28,3,<NA>,
321,2024-12-30,1,<NA>,12.28


In [37]:
dates_df

,Date,type,title_id,from_day,status,year,year_iso,quarter,month,week_iso,day_of_month,day_of_week
0,2013-01-01,1,1,,Праздничный,2013,2013,1,1,1,1,2
1,2013-01-02,1,1,,Праздничный,2013,2013,1,1,1,2,3
2,2013-01-03,1,1,,Праздничный,2013,2013,1,1,1,3,4
3,2013-01-04,1,1,,Праздничный,2013,2013,1,1,1,4,5
4,2013-01-05,1,1,,Праздничный,2013,2013,1,1,1,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4378,2024-12-27,<NA>,<NA>,NaN,Рабочий,2024,2024,4,12,52,27,5
4379,2024-12-28,3,<NA>,,Рабочий,2024,2024,4,12,52,28,6
4380,2024-12-29,<NA>,<NA>,NaN,Выходной,2024,2024,4,12,52,29,7
4381,2024-12-30,1,<NA>,12.28,Праздничный,2024,2025,4,12,1,30,1
